In [ ]:
###
# Plot of Noam Annealing
###

import matplotlib.pyplot as plt
%matplotlib inline
d_model = 256
warmup = 4000
max_steps = 20000
lr = 1
min_lr = 1e-5

def _noam_annealing(initial_lr, step):
    normalize = d_model ** (-0.5)
    warmup_steps = warmup
    mult = normalize * min(step ** (-0.5), step * (warmup_steps ** (-1.5)))
    out_lr = initial_lr * mult
    if step > warmup_steps:
        out_lr = max(out_lr, min_lr)
    return out_lr

x = range(1, max_steps)
y = []
for i in x:
    y.append(_noam_annealing(lr, i))
    if i % 1000 == 0:
        print(_noam_annealing(lr, i))
plt.plot(x, y)
plt.show()

class NoamAnnealing(object):
    def __init__(
        self, d_model, max_steps, warmup_steps=None, min_lr=0.0, last_epoch=-1
    ):
        self._normalize = d_model ** (-0.5)

        self.max_steps = max_steps
        self.warmup_steps = warmup_steps
        self.min_lr = min_lr

    def get_lr(self, step, lr):
        if step > self.max_steps:
            return [self.min_lr for _ in self.base_lrs]

        new_lrs = self._noam_annealing(initial_lr=lr, step=step)
        return new_lrs

    def _noam_annealing(self, initial_lr, step):
        mult = self._normalize * min(step ** (-0.5), step * (self.warmup_steps ** (-1.5)))
        out_lr = initial_lr * mult
        if step > self.warmup_steps:
            out_lr = max(out_lr, self.min_lr)
        return out_lr

d_model=256
max_steps=20000
warmup_steps=4000
min_lr=1e-5
lr = 1

myanneal = NoamAnnealing(d_model=d_model, max_steps=max_steps, warmup_steps=warmup_steps, min_lr=min_lr)
x = range(1, max_steps)
y = []
for i in x:
    y.append(myanneal.get_lr(i, lr))
    if i % 1000 == 0:
        print(myanneal.get_lr(i, lr))
plt.plot(x, y)
plt.show()

In [ ]:
###
# F0 Checks
###

from pathlib import Path
import IPython.display as ipd
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import librosa
import librosa.display
import json

%matplotlib inline

audio = []
all_data = []
total_duration = 0
with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/manifest35.json", "r") as f:
    lines = f.readlines()
    np.random.shuffle(lines)
    for lin in lines:
        data = json.loads(lin)
        total_duration += data['duration']
        if total_duration > 45*60:
            break
        audio.append(data['audio_filepath'])
        all_data.append(data)
print(f"Total duration: {total_duration}")

with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/manifest35_45mins.json", "w") as f:
    for i, line in enumerate(all_data):
        f.write(json.dumps(line)+'\n')

# audio = []
# total_duration = 0
# with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/manifest35_15mins.json", "r") as f:
#     lines = f.readlines()
#     for lin in lines:
#         data = json.loads(lin)
#         audio.append(data['audio_filepath'])
    

# pitch_fmin = 64 # REALLY Should change this
# pitch_fmax = 512 # Should change this
# n_window_size=2048  # Don't have to change this
# # all_f0 = np.array([])
# for i, f in enumerate(audio):
#     y, _ = librosa.core.load(f, sr=44100)
#     stft_matrix = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=2048, window="hann")))
#     f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=pitch_fmin, fmax=pitch_fmax, frame_length=n_window_size, sr=44100, fill_na=0.)
    
#     ## UNCOMMENT THIS SECTION TO EXPERIMENT WITH FMIN, FMAX
# #     fig, ax = plt.subplots()
# #     librosa.display.specshow(stft_matrix, sr=44100, hop_length=512, y_axis="log", ax=ax)
# #     ax.plot(f0, label='f0', color='cyan', linewidth=3)

#     # Hack to remove off_pitches
#     found_flat_area = False
#     start = 0
#     end = 0
#     for i, pitch in enumerate(f0):
#         if pitch==0.:
#             if found_flat_area:
#                 end = i
#                 # Decide whether to trim this
#                 voice_prob_avg = np.mean(voiced_probs[start:end])
#                 if voice_prob_avg < 0.03:
#                     # Delete pitch
#                     f0[start:end] = 0.
#                 else:
#                     # Probably started real audio
#                     break
#                 found_flat_area = False
#                 start = 0
#             continue
#         elif not found_flat_area:
#             found_flat_area = True
#             start = i
#             end = 0
#     found_flat_area = False
#     start = 0
#     end = 0
#     for i, pitch in enumerate(reversed(f0)):
#         if pitch==0.:
#             if found_flat_area:
#                 end = i
#                 # Decide whether to trim this
#                 voice_prob_avg = np.mean(voiced_probs[(len(f0)-end):(len(f0)-start)])
#                 if voice_prob_avg < 0.03:
#                     # Delete pitch
#                     f0[(len(f0)-end):(len(f0)-start)] = 0.
#                 else:
#                     # Probably started real audio
#                     break
#                 found_flat_area = False
#                 start = 0
#             continue
#         elif not found_flat_area:
#             found_flat_area = True
#             start = i
#             end = 0

# ## UNCOMMENT THIS SECTION TO EXPERIMENT WITH FMIN, FMAX
#     all_f0 = np.concatenate((all_f0,f0[f0!=0].flatten()))
#     ax.plot(f0, label='f0', color='cyan', linewidth=2)
#     ax.plot(voiced_probs*500, label='f0', linewidth=3)
# #     plt.show()

# # ### UNCOMMENT THIS SECTION WHEN YOU FINALIZE FMIN, FMAX
# #     filename = f"/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/pitch_prior/{Path(f).stem}_pitch_pyin_fmin{pitch_fmin}_fmax{pitch_fmax}_fl{n_window_size}.npy"
# #     np.save(filename, f0)

# ## WE STILL NEED TO FIND MEAN AND STD FOR EACH 15min, 30min, etc. manifest
# print(f"pitch mean: {np.mean(all_f0)}")
# print(f"pitch std: {np.std(all_f0)}")
    
# # import torch

# # y, _ = librosa.core.load("/data/speech/LJSpeech/wavs/LJ030-0200.wav", sr=22050)
# # stft_matrix = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=1024, window="hann", hop_length=256)))
# # %matplotlib inline
# # fig, ax = plt.subplots()
# # librosa.display.specshow(stft_matrix, sr=22050, hop_length=256, y_axis="log", ax=ax)
# # f0 = torch.load("/data/speech/LJSpeech/supplementary/LJ030-0200_melodia_f0min80_f0max800_harm1.0_mps0.0.pt")["f0"]
# # f0 = f0.numpy().squeeze()
# # f0[f0==0] = np.nan
# # ax.plot(f0, label='f0', color='cyan', linewidth=3)

# # import pysptk

# # y, _ = librosa.core.load("/data/speech/LJSpeech/wavs/LJ030-0200.wav", sr=22050)
# # stft_matrix = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=1024, window="hann", hop_length=256)))
# # %matplotlib inline
# # fig, ax = plt.subplots()
# # librosa.display.specshow(stft_matrix, sr=22050, hop_length=256, y_axis="log", ax=ax)
# # f0 = pysptk.rapt(y.astype(np.float32) * 32768, fs=22050, hopsize=256, otype="f0")
# # f0[f0==0] = np.nan
# # ax.plot(f0, label='f0', color='cyan', linewidth=3)

# # librosa.display.specshow(stft_matrix, sr=22050, hop_length=256, y_axis="log")

In [ ]:
###
# GL
###

import librosa
def griffin_lim(magnitudes, n_iters=50, n_fft=1024):
    """
    Griffin-Lim algorithm to convert magnitude spectrograms to audio signals
    """
    phase = np.exp(2j * np.pi * np.random.rand(*magnitudes.shape))
    complex_spec = magnitudes * phase
    signal = librosa.istft(complex_spec)
    if not np.isfinite(signal).all():
        logging.warning("audio was not finite, skipping audio saving")
        return np.array([0])

    for _ in range(n_iters):
        _, phase = librosa.magphase(librosa.stft(signal, n_fft=n_fft))
        complex_spec = magnitudes * phase
        signal = librosa.istft(complex_spec)
    return signal

linear = spec.cpu().numpy().squeeze(0)
linear = np.dot(librosa.filters.mel(44100, 2048, n_mels=80, fmin=0.0, fmax=None).T, linear)
linear = np.clip(linear, a_min=0, a_max=255)
audio = griffin_lim(linear**1.5, n_fft=2048)

ipd.Audio(audio, rate=44100)

In [ ]:
###
# Inference (T2+HiFiGAN)
###

import numpy as np
import torch
from scipy.io.wavfile import write


from nemo.collections.asr.data.audio_to_text import _AudioTextDataset
from nemo.collections.tts.models import Tacotron2Model, FastPitchModel
from nemo.collections.tts.modules.hifigan_modules import Generator


# t2 = Tacotron2Model.restore_from("/home/jasoli/nemo/NeMo/examples/tts/Tacotron2.nemo")
# t2 = Tacotron2Model.restore_from("/home/jasoli/nemo/NeMo/Tacotron2-Helen-char.nemo")
t2 = Tacotron2Model.restore_from("/home/jasoli/nemo/NeMo/Tacotron2-large-9017.nemo")
t2 = t2.cuda().eval()
t2.decoder.gate_threshold = 0.4
labels = t2.cfg.labels

vocoder = Generator(
    resblock=1,
    upsample_rates=[8, 8, 4, 2],
    upsample_kernel_sizes=[16, 16, 4, 4],
    upsample_initial_channel=512,
    resblock_kernel_sizes=[3, 7, 11],
    resblock_dilation_sizes=[[1, 3, 5], [1, 3, 5], [1, 3, 5]],
)
nemo_gen_keys = [k for k in vocoder.state_dict().keys()]
# adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/Hifigan_ADLR_male_44k_00632500")
adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/Hifigan_ADLR_male_44k_00632500.pt")["generator"]
# adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/Hifigan_ADLR_Helen_44k_00570000.pt")["generator"]
adlr_gen_keys = adlr_gen_ckpt.keys()

new_nemo_ckpt = {nemo_key: adlr_gen_ckpt[adlr_key] for adlr_key, nemo_key in zip(adlr_gen_keys, nemo_gen_keys)}
vocoder.load_state_dict(new_nemo_ckpt)
vocoder = vocoder.cuda().eval().half()

dataset = _AudioTextDataset(
     "/data/speech/HiFiTTS/9017_manifest_clean_dev.json",
#     "/data/speech/Helen/helen_val.json",
#     "/mnt/ssd1/data/LJSpeech-1.1/nvidia_ljspeech_test.json",
    parser=t2.parser,
    sample_rate=44100,
    bos_id=len(labels),
    eos_id=len(labels) + 1,
    pad_id=len(labels) + 2,
    max_utts=5,
    min_duration=1.,
    max_duration=16.,
)
dataloader = torch.utils.data.DataLoader(
    dataset=dataset, batch_size=1, collate_fn=dataset.collate_fn, num_workers=0, shuffle=False
)
device = torch.device("cuda")
all_utterances = 0
all_samples = 0

batches = []
for b in dataloader:
    batches.append(b)

def infer_vocoder(model, spec: torch.Tensor):
    audio = model(x=spec).squeeze(1)
    return audio

audios = []
gt_audios = []
for batch in batches:
    gt_audio, _, text, text_length = batch
    gt_audios.append(gt_audio)
    text = text.to(device)
    text_length = text_length.to(device)
    with torch.no_grad():
        spectrogram = t2.generate_spectrogram(tokens=text)
        audios.append(infer_vocoder(model=vocoder, spec=spectrogram.half()))

import IPython.display as ipd
for i, (aud, gt_aud) in enumerate(zip(audios, gt_audios)):
    print(f"Sample_{i}")
    ipd.display(ipd.Audio(aud.float().cpu().numpy(), rate=44100))
    ipd.display(ipd.Audio(gt_aud.float().cpu().numpy(), rate=44100))
    write(f"sample_{i}.wav", 44100, aud.float().cpu().numpy()[0])

In [ ]:
###
# Splicing rap mp3  - did not work
###
import librosa
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import numpy as np
import librosa.display
import soundfile as sf


y, sr = librosa.core.load("./3899_Eminem - The Real Slim Shady (Acapella).mp3", sr=None)

start = int((60+21.7)*sr)
# end = int((60+30.58)*sr)  # end of please stand up x3
end = int((60+28.2)*sr)  # end of first please stand up 
spliced = y[start:end]
ipd.display(ipd.Audio(spliced, rate=sr))

stft_matrix = librosa.amplitude_to_db(np.abs(librosa.stft(spliced, n_fft=2048, window="hann")))
f0, voiced_flag, voiced_probs = librosa.pyin(spliced, fmin=120, fmax=512, frame_length=2048, sr=sr)
%matplotlib inline
fig, ax = plt.subplots(figsize=(12, 6))
librosa.display.specshow(stft_matrix, sr=sr, hop_length=512, y_axis="log", ax=ax)
ax.plot(f0, label='f0', color='cyan', linewidth=3)

sf.write('chorus.wav', spliced, sr)

f0_mean = np.mean(f0[~np.isnan(f0)])
f0_std = np.std(f0[~np.isnan(f0)])

f0_stand = (f0 - f0_mean)/f0_std
f0_stand[np.isnan(f0_stand)] = 0.

# Load fastpitch model

from nemo.collections.tts.models import FastPitchModel
import torch
from nemo.collections.tts.torch.helpers import beta_binomial_prior_distribution
from nemo.utils import logging
logging.setLevel(logging.ERROR)
import librosa
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import numpy as np
import librosa.display
import soundfile as sf

fastpitch = FastPitchModel.restore_from("/home/jasoli/nemo/NeMo/FastPitch-Align-8051.nemo").eval()

# inp = ("I'm Slim Shady, yes, I'm the real Shady. "
#        "All you other Slim Shadys are just imitating. "
#        "So won't the real Slim Shady please stand up?")
# tokens = fastpitch.parse(inp)

# mels, spec_len = fastpitch.preprocessor(input_signal=torch.tensor(spliced).unsqueeze(0).cuda(), length=torch.tensor([len(spliced)]).cuda())

# mels_pred, _, durs_predicted, log_durs_pred, pitch_pred, attn_soft, attn_logprob, attn_hard, attn_hard_dur, pitch = fastpitch(
#     text=tokens,
#     durs=None,
#     pitch=None,
#     speaker=None,
#     pace=1.0,
#     spec=mels,
#     attn_prior=torch.tensor(beta_binomial_prior_distribution(tokens.shape[1],mels.shape[2])).unsqueeze(0).cuda(),
#     mel_lens=spec_len,
#     input_lens=torch.tensor(tokens.shape[1]).unsqueeze(0),
# )

# spec, *_ = fastpitch(
#     text=tokens,
#     durs=new_dur.cuda(),
#     pitch=torch.tensor(f0_stand).cuda().unsqueeze(0),
#     speaker=None,
#     pace=1.0,
#     spec=None,
#     attn_prior=None,
#     mel_lens=None,
#     input_lens=torch.tensor(tokens.shape[1]).unsqueeze(0),
# )

# spec_2, *_ = fastpitch(
#     text=tokens,
#     durs=attn_hard_dur,
#     pitch=None,
#     speaker=None,
#     pace=1.0,
#     spec=None,
#     attn_prior=None,
#     mel_lens=None,
#     input_lens=torch.tensor(tokens.shape[1]).unsqueeze(0),
# )

fig, ax = plt.subplots(figsize=[15,8])
ax.imshow(mels.cpu().detach().numpy()[0], origin="lower")
plt.show()

from nemo.collections.tts.modules.hifigan_modules import Generator

vocoder = Generator(
    resblock=1,
    upsample_rates=[8, 8, 4, 2],
    upsample_kernel_sizes=[16, 16, 4, 4],
    upsample_initial_channel=512,
    resblock_kernel_sizes=[3, 7, 11],
    resblock_dilation_sizes=[[1, 3, 5], [1, 3, 5], [1, 3, 5]],
)
nemo_gen_keys = [k for k in vocoder.state_dict().keys()]
adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/hifigan_gen_nemo_8051_00524000.pt")
adlr_gen_keys = adlr_gen_ckpt.keys()

new_nemo_ckpt = {nemo_key: adlr_gen_ckpt[adlr_key] for adlr_key, nemo_key in zip(adlr_gen_keys, nemo_gen_keys)}
vocoder.load_state_dict(new_nemo_ckpt)
vocoder = vocoder.cuda().eval()


audio = vocoder(x=spec)
audio2 = vocoder(x=mels_pred)

ipd.display(ipd.Audio(audio.cpu().detach().numpy()[0], rate=44100))
ipd.display(ipd.Audio(audio2.cpu().detach().numpy()[0], rate=44100))

new_dur = torch.zeros(attn_hard_dur.shape)

for i in range(attn_hard_dur.shape[1]):
    if tokens[0,i] == 0:
        new_dur[0,i] = int(attn_hard_dur[0,i] / 1.3)
    else:
        new_dur[0,i] = int(attn_hard_dur[0,i] / 0.9)

In [ ]:
###
# Fastpitch female 2 male attempt 1 - did not work
###

from nemo.collections.tts.models import FastPitchModel
import torch
from nemo.collections.tts.torch.helpers import beta_binomial_prior_distribution
from nemo.utils import logging
logging.setLevel(logging.ERROR)
import librosa
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import numpy as np
import librosa.display
import soundfile as sf

fastpitch = FastPitchModel.restore_from("/home/jasoli/nemo/NeMo/FastPitch-Align-8051.nemo").eval().cuda()
from nemo.collections.tts.modules.hifigan_modules import Generator

vocoder = Generator(
    resblock=1,
    upsample_rates=[8, 8, 4, 2],
    upsample_kernel_sizes=[16, 16, 4, 4],
    upsample_initial_channel=512,
    resblock_kernel_sizes=[3, 7, 11],
    resblock_dilation_sizes=[[1, 3, 5], [1, 3, 5], [1, 3, 5]],
)
nemo_gen_keys = [k for k in vocoder.state_dict().keys()]
adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/hifigan_gen_nemo_8051_00524000.pt")
adlr_gen_keys = adlr_gen_ckpt.keys()

new_nemo_ckpt = {nemo_key: adlr_gen_ckpt[adlr_key] for adlr_key, nemo_key in zip(adlr_gen_keys, nemo_gen_keys)}
vocoder.load_state_dict(new_nemo_ckpt)
vocoder = vocoder.cuda().eval()

inp = ("Who do I sound like?")
tokens = fastpitch.parse(inp)

mels_pred, _, durs_predicted, log_durs_pred, pitch_pred, attn_soft, attn_logprob, attn_hard, attn_hard_dur, pitch = fastpitch(
    text=tokens,
    durs=None,
    pitch=None,
    speaker=None,
    pace=1.0,
    spec=None,
    attn_prior=None,
    mel_lens=None,
    input_lens=torch.tensor(tokens.shape[1]).unsqueeze(0),
)
fig, ax = plt.subplots(figsize=[15,8])
ax.imshow(mels_pred.cpu().detach().numpy()[0], origin="lower")
plt.show()

audio = vocoder(x=mels_pred)

ipd.display(ipd.Audio(audio.cpu().detach().numpy()[0], rate=44100))

print(durs_predicted)
mels_pred, _, durs_predicted, log_durs_pred, pitch_pred, attn_soft, attn_logprob, attn_hard, attn_hard_dur, pitch = fastpitch(
    text=tokens,
    durs=None,
    pitch=None,
    speaker=None,
    pace=0.4,
    spec=None,
    attn_prior=None,
    mel_lens=None,
    input_lens=torch.tensor(tokens.shape[1]).unsqueeze(0).cuda(),
    pitch_transform=-50,
)
fig, ax = plt.subplots(figsize=[15,8])
ax.imshow(mels_pred.cpu().detach().numpy()[0], origin="lower")
plt.show()

audio = vocoder(x=mels_pred)

ipd.display(ipd.Audio(audio.cpu().detach().numpy()[0], rate=44100))

In [ ]:
###
# HiFigan cycle stationary consistency
###

import librosa
import IPython.display as ipd
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import numpy as np
import librosa.display

y, sr = librosa.core.load("/home/jasoli/example.wav", sr=None)
stft_matrix = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=2048, window="hann")))

fig, ax = plt.subplots(figsize=[15,8])
librosa.display.specshow(stft_matrix, sr=sr, hop_length=512, y_axis="log", ax=ax)
ipd.display(ipd.Audio(y, rate=44100))
plt.show()

In [ ]:
###
# TorchScript export
###
from nemo.collections.tts.models import FastPitchModel, HifiGanModel

fp = FastPitchModel.from_pretrained("tts_en_fastpitch").cpu()
# fp.export("fastpitch_ngc_ljs_pitch.ts")
# hf = HifiGanModel.from_pretrained("tts_hifigan").cpu()
# hf.export("tts_hifigan.ts")

# import torch

# fp_ts = torch.jit.load('fastpitch_ngc_ljs_pitch.ts')

# tokens = fp.parse("Hello.")
# # spec = fp_ts(tokens)
# print(fp._parser._labels_map)
# with open("fastpitch_ngc_ljs_mappings.txt", "w") as f:
#     for key, value in fp._parser._labels_map.items():
#         f.write(f"{value} {key}")

In [ ]:
###
# ADLR checkpoint to .NeMo (only generator)
###

import numpy as np
import torch
from scipy.io.wavfile import write


from nemo.collections.tts.models import HifiGanModel
from nemo.collections.tts.modules.hifigan_modules import Generator

hf_conf = {}
hf_conf["generator"] = {
    "_target_":"nemo.collections.tts.modules.hifigan_modules.Generator",
    "resblock":1,
    "upsample_rates":[8, 8, 4, 2],
    "upsample_kernel_sizes":[16, 16, 4, 4],
    "upsample_initial_channel":512,
    "resblock_kernel_sizes":[3, 7, 11],
    "resblock_dilation_sizes":[[1, 3, 5], [1, 3, 5], [1, 3, 5]],
}
hf = HifiGanModel(hf_conf)

# vocoder = Generator(
#     resblock=1,
#     upsample_rates=[8, 8, 4, 2],
#     upsample_kernel_sizes=[16, 16, 4, 4],
#     upsample_initial_channel=512,
#     resblock_kernel_sizes=[3, 7, 11],
#     resblock_dilation_sizes=[[1, 3, 5], [1, 3, 5], [1, 3, 5]],
# )
nemo_gen_keys = [k for k in hf.state_dict().keys()]
adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/Hifigan_ADLR_Helen_44k_00570000.pt")["generator"]
adlr_gen_keys = adlr_gen_ckpt.keys()

new_nemo_ckpt = {nemo_key: adlr_gen_ckpt[adlr_key] for adlr_key, nemo_key in zip(adlr_gen_keys, nemo_gen_keys)}
hf.load_state_dict(new_nemo_ckpt)
hf.save_to("Hifigan_ADLR_Helen.nemo")

In [ ]:
###
# ADLR checkpoint to .NeMo (generator+disc)
###
from collections import OrderedDict

import torch
from omegaconf import OmegaConf

from nemo.collections.tts.models import HifiGanModel


adlr_gen_ckpt = torch.load(
    "/home/jasoli/nemo/NeMo/g_00587500"
)
adlr_dis_ckpt = torch.load(
    "/home/jasoli/nemo/NeMo/do_00587500"
)
save_path = "/home/jasoli/nemo/NeMo/checkpoints/hifigan_from_adlr_in_nemo_helen_00587500.nemo"
model_config = OmegaConf.load('/home/jasoli/nemo/NeMo/examples/tts/my-hifigan-44k.yaml')

del model_config["model"]["train_ds"]
del model_config["model"]["validation_ds"]
del model_config["exp_manager"]

model = HifiGanModel(cfg=model_config.model)
nemo_gen_keys = [k for k in model.state_dict().keys() if "generator" in k]
adlr_gen_keys = adlr_gen_ckpt["generator"].keys()

new_nemo_ckpt = {
    nemo_key: adlr_gen_ckpt["generator"][adlr_key] for adlr_key, nemo_key in zip(adlr_gen_keys, nemo_gen_keys)
}

for k in model.state_dict().keys():
    if "msd.discriminators" in k:
        new_nemo_ckpt[k] = adlr_dis_ckpt["msd"][k.replace("msd.", "")]
    elif "mpd.discriminators" in k:
        new_nemo_ckpt[k] = adlr_dis_ckpt["mpd"][k.replace("mpd.", "")]

model.load_state_dict(OrderedDict(new_nemo_ckpt), strict=False)
model.save_to(save_path)


In [ ]:
###
# Inference with FastPitch over dataset to prepare for hifigan finetuning
###

import argparse
import json
from operator import ne
from pathlib import Path

import numpy as np
import torch

from nemo.collections.asr.data.audio_to_text import AudioToCharWithPriorAndPitchDataset
from nemo.collections.tts.models import FastPitchModel

def generate_samples():
    prior_folder = "/data/speech/Helen/FasPitch_align/"
    nemo_checkpoint = "/home/jasoli/nemo/NeMo/checkpoints/FastPitch-Align-Helen.nemo"
    manifest_filepath = "/data/speech/Helen/helen_train.json"
#     manifest_filepath = "/data/speech/Helen/helen_val.json"
    syn_mel_folder = Path("/data/speech/Helen/FastPitch_align_syn_mel")
    
    model = FastPitchModel.restore_from(nemo_checkpoint, map_location="cpu").cuda(1).eval()
    vocab_dict = {
        "notation": "phonemes",
        "punct": True,
        "spaces": True,
        "stresses": True,
        "add_blank_at": None,
        "pad_with_space": True,
        "chars": True,
        "improved_version_g2p": True,
    }
    dataset = AudioToCharWithPriorAndPitchDataset(
        sup_data_path=prior_folder,
        n_window_stride=512,
        n_window_size=2048,
        manifest_filepath=manifest_filepath,
        sample_rate=44100,
        pitch_fmin=model._cfg.pitch_fmin,
        pitch_fmax=model._cfg.pitch_fmax,
        pitch_avg=model._cfg.pitch_avg,
        pitch_std=model._cfg.pitch_std,
        vocab=vocab_dict,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset=dataset, batch_size=32, collate_fn=dataset.collate_fn, num_workers=8, shuffle=False
    )

    if not syn_mel_folder.exists():
        syn_mel_folder.mkdir()

    i = 0
    for batch in dataloader:
        audio, audio_lens, text, text_lens, attn_prior, _, _ = batch
        with torch.no_grad():
            mels, spec_len = model.preprocessor(input_signal=audio.cuda(1), length=audio_lens.cuda(1))
            mels_pred, _, _, _, _, _, _, _, attn_hard_dur, _ = model(
                text=text.cuda(1),
                durs=None,
                pitch=None,
                speaker=None,
                pace=1.0,
                spec=mels,
                attn_prior=attn_prior.cuda(1),
                mel_lens=spec_len,
                input_lens=text_lens.cuda(1),
            )
        mels = mels_pred.cpu().numpy()
        for j, mel in enumerate(mels):
            mel_length = int(torch.sum(attn_hard_dur[j]))
            second_half = "train_{}" if "train" in manifest_filepath else "val_{}"
            save_path = syn_mel_folder / second_half.format(i)
            np.save(save_path, mel[:, :mel_length])
            if i < 10:
                print(save_path)
            i += 1


def make_new_json():
    manifest_filepath = "/data/speech/Helen/helen_train.json"
    new_json_filepath = "/data/speech/Helen/helen_train_synmel.json"
#     manifest_filepath = "/data/speech/Helen/helen_val.json"
#     new_json_filepath = "/data/speech/Helen/helen_val_synmel.json"
    syn_mel_folder = Path("/data/speech/Helen/FastPitch_align_syn_mel")

    with open(manifest_filepath, "r") as data_in:
        lines = data_in.readlines()

    with open(new_json_filepath, "w") as data_out:
        for i, line in enumerate(lines):
            in_line = json.loads(line)
            second_half = "train_{}.npy" if "train" in manifest_filepath else "val_{}.npy"
            save_path = syn_mel_folder / second_half.format(i)
            in_line["mel_filepath"] = str(save_path)
            data_out.write(json.dumps(in_line) + '\n')

generate_samples()
make_new_json()

In [ ]:
### Hifigan finetuning

import json

# with open("/data/speech/Helen/helen_train.json", "r") as data_in:
with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/manifest35_15mins.json", "r") as data_in:
    lines = data_in.readlines()

# with open("/data/speech/Helen/helen_train_synmel.json", "w") as data_out:
with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/manifest35_syn_15mins.json", "w") as data_out:
    for i, line in enumerate(lines):
        in_line = json.loads(line)
        # in_line["mel_filepath"] = f"/data/speech/Helen/FastPitch_align_syn_mel/train_{i}.npy"
        in_line["mel_filepath"] = f"/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/audio_syn_15min/train_{i}.npy"
        data_out.write(json.dumps(in_line)+'\n')

In [ ]:
###
# Inference (FP+HiFiGAN)
###
import numpy as np
import torch
from scipy.io.wavfile import write


from nemo.collections.asr.data.audio_to_text import _AudioTextDataset
from nemo.collections.tts.models import FastPitchModel, HifiGanModel
from nemo.collections.tts.modules.hifigan_modules import Generator


fp = FastPitchModel.restore_from("/home/jasoli/nemo/NeMo/checkpoints/Helen/FastPitch-Align-Helen.nemo", map_location="cpu")
# fp = FastPitchModel.restore_from("/home/jasoli/nemo/NeMo/FinetuningDemo/Sirisha_15_mins/FastPitch/2021-09-28_22-20-50/checkpoints/FastPitch.nemo")
# fp = FastPitchModel.load_from_checkpoint("/home/jasoli/nemo/NeMo/checkpoints/JonCkpts/FastPitchNoMixJon.ckpt")
fp = fp.cuda(1).eval()

# vocoder = Generator(
#     resblock=1,
#     upsample_rates=[8, 8, 4, 2],
#     upsample_kernel_sizes=[16, 16, 4, 4],
#     upsample_initial_channel=512,
#     resblock_kernel_sizes=[3, 7, 11],
#     resblock_dilation_sizes=[[1, 3, 5], [1, 3, 5], [1, 3, 5]],
# )
# nemo_gen_keys = [k for k in vocoder.state_dict().keys()]
# # adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/Hifigan_ADLR_male_44k_00632500")
# # adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/Hifigan_ADLR_male_44k_00632500")["generator"]
# adlr_gen_ckpt = torch.load("/home/jasoli/nemo/NeMo/Hifigan_ADLR_Helen_44k_00570000.pt")["generator"]
# adlr_gen_keys = adlr_gen_ckpt.keys()

# new_nemo_ckpt = {nemo_key: adlr_gen_ckpt[adlr_key] for adlr_key, nemo_key in zip(adlr_gen_keys, nemo_gen_keys)}
# vocoder.load_state_dict(new_nemo_ckpt)

# vocoder = HifiGanModel.load_from_checkpoint("/home/jasoli/nemo/NeMo/checkpoints/JonCkpts/HifiGanMix.ckpt")
vocoder = HifiGanModel.restore_from("/home/jasoli/nemo/NeMo/checkpoints/hifiganfinetune/HifiGan/2021-09-17_21-15-08/checkpoints/HifiGan.nemo", map_location="cpu")
# whatever = torch.load("/home/jasoli/nemo/NeMo/HifiGan--val_loss=0.26-epoch=246-last.ckpt")
# vocoder = HifiGanModel(cfg=whatever["hyper_parameters"])
# vocoder.load_state_dict(whatever["state_dict"])
vocoder = vocoder.cuda(1).eval().half()

vocoder2 = HifiGanModel.restore_from("/home/jasoli/nemo/NeMo/checkpoints/hifiganfinetune_try2/HifiGan/2021-09-29_20-54-05/checkpoints/HifiGan.nemo", map_location="cpu")
vocoder2 = vocoder.cuda(1).eval().half()


import json
import soundfile as sf
import IPython.display as ipd

with open("/data/speech/LJSpeech/nvidia_ljspeech_test.json", "r") as f:
    lines = f.readlines()
lines = [json.loads(l) for l in lines]

for i, l in enumerate(lines):
    tokens = fp.parse(l['text']).cuda(1)
    with torch.no_grad():
        spectrogram = fp.generate_spectrogram(tokens=tokens)
    #     audio = vocoder(x=spectrogram.half()).squeeze(1)  # x for generator
        audio = vocoder(spec=spectrogram.half()).squeeze(1)  # spec for hifiganmodel
        audio2 = vocoder2(spec=spectrogram.half()).squeeze(1)  # spec for hifiganmodel
    print(i)
    print((audio==audio2).all())
    ipd.display(ipd.Audio(audio.float().cpu().numpy(), rate=44100))
    ipd.display(ipd.Audio(audio2.float().cpu().numpy(), rate=44100))
#     sf.write(f'sample_{i}.wav', audio.float().cpu().numpy().squeeze(0), 44100)
    if i > 10:
        break



In [ ]:
###
# Mix Sally duration with Helen model
###
from copy import deepcopy
import torch
from omegaconf import OmegaConf

from nemo.collections.tts.models import FastPitchModel
from nemo.core.config import hydra_runner
import IPython.display as ipd
from nemo.core.classes import typecheck

typecheck.set_typecheck_enabled(enabled=False)


config = FastPitchModel.restore_from("/home/jasoli/nemo/NeMo/checkpoints/Helen/FastPitch-Align-Helen.nemo", return_config=True)
config.vocab = config.train_ds.dataset.vocab
del config.train_ds
del config.validation_ds
fastpitch = FastPitchModel(cfg=config)
sally_checkpoint = torch.load("/home/jasoli/nemo/NeMo/checkpoints/Sally/model_weights.ckpt")
helen_checkpoint = torch.load("/home/jasoli/nemo/NeMo/checkpoints/Helen/model_weights.ckpt")

# print(config.train_ds.dataset)
model_params = {}
for n, p in fastpitch.named_parameters():
    model_params[n] = ''
for key, value in sally_checkpoint.items():
    if 'encoder' in key:
        new_key_pos = key.find("encoder")
        new_key = key[:new_key_pos+len("encoder")]+"_sally"+key[new_key_pos+len("encoder"):]
        model_params[new_key] = value
    elif 'fastpitch.duration_predictor' in key:
        new_key_pos = key.find("duration_predictor")
        new_key = key[:new_key_pos+len("duration_predictor")]+"_sally"+key[new_key_pos+len("duration_predictor"):]
        model_params[new_key] = value
for key, value in helen_checkpoint.items():
    if 'encoder' in key:
        new_key_pos = key.find("encoder")
        new_key = key[:new_key_pos+len("encoder")]+"_helen"+key[new_key_pos+len("encoder"):]
        model_params[new_key] = value
    elif 'fastpitch.duration_predictor' in key:
        new_key_pos = key.find("duration_predictor")
        new_key = key[:new_key_pos+len("duration_predictor")]+"_helen"+key[new_key_pos+len("duration_predictor"):]
        model_params[new_key] = value
    else:
        model_params[key] = value

fastpitch.load_state_dict(model_params)
fastpitch.save_to("/home/jasoli/nemo/NeMo/checkpoints/Fastpitch-Align-SallySpeed-HelenVoice.nemo")

## INFERENCE
from nemo.collections.tts.models import HifiGanModel
vocoder = HifiGanModel.restore_from("/home/jasoli/nemo/NeMo/checkpoints/hifigan_from_adlr_in_nemo_helen_00587500.nemo")
# vocoder = HifiGanModel.restore_from("/home/jasoli/nemo/NeMo/checkpoints/hifigan_adlr_universal_44k.nemo")
# whatever = torch.load("/home/jasoli/nemo/NeMo/HifiGan--val_loss=0.26-epoch=246-last.ckpt")
# vocoder = HifiGanModel(cfg=whatever["hyper_parameters"])
# vocoder.load_state_dict(whatever["state_dict"])

vocoder = vocoder.cuda().eval().half()
fastpitch = fastpitch.cuda().eval()

tokens = fastpitch.parse("Why do you think I would say something like that?").cuda()
with torch.no_grad():
    for i in [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        spectrogram = fastpitch.generate_spectrogram(tokens=tokens, helen_mix=i)
    #     audio = vocoder(x=spectrogram.half()).squeeze(1)  # x for generator
        audio = vocoder(spec=spectrogram.half()).squeeze(1)  # spec for hifiganmodel
        print(i)
        ipd.display(ipd.Audio(audio.float().cpu().numpy(), rate=44100))

In [ ]:
### TTS Finetuning

from pathlib import Path
import librosa
import IPython.display as ipd
import random
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import numpy as np
import librosa.display

filelist = Path("/data/speech/NvidiaCustomTTSFinetuning/Sirisha").glob("*.wav")
filelist = [t for t in filelist]
random.shuffle(filelist)
for f in filelist:
    y, sr = librosa.load(f, sr=None)
    trimmed, _ = librosa.effects.trim(y, top_db=60, frame_length=1024, hop_length=256)
    print(len(y)-len(trimmed))
    assert sr == 44100
    ipd.display(ipd.Audio(trimmed, rate=sr))
    mag = np.abs(librosa.stft(trimmed, n_fft=2048, window="hann"))
    stft_matrix = librosa.amplitude_to_db(mag)
    fig, ax = plt.subplots(figsize=[8,6])
    librosa.display.specshow(stft_matrix, sr=sr, hop_length=512, y_axis="log", ax=ax)
    plt.show()
    energy = np.sum(stft_matrix, axis=1)
    np.sum(mag, axis=0)
    plt.plot(np.sum(mag, axis=0))
    break

In [ ]:
### VCTK reading

from pathlib import Path

harvard_sentences = set()
with open("/home/jasoli/harvard.txt", "r") as f:
    for line in f.readlines():
        harvard_sentences.add(line)
        
VCTK_sentences = set()
for file in Path("/data/speech/VCTK/txt").glob("*/*.txt"):
    with open(file) as f:
        text = f.read()

#     file_number = file.stem.split("_")[1]
#     if file_number in VCTK_sentences:
#         assert text == VCTK_sentences[file_number]["data"], f"{text}!={VCTK_sentences[file_number]['data']}|{file}|{VCTK_sentences[file_number]['file']}"
#     else:
#         VCTK_sentences[file_number] = {'data': text, 'file': file}
    VCTK_sentences.add(text.strip())
    assert text not in harvard_sentences, f"{file}|{text}| was in harvard"

sentences = sorted(VCTK_sentences)
# sentence_length = [len(s) for s in sentences]
# hist = {}
# for l in sentence_length:
#     if l not in hist:
#         hist[l] = 1
#     else:
#         hist[l] += 1
filtered_sentences = []
for s in sentences:
    if len(s) > 130 or len(s) < 20:
        continue
    if any([c.isupper() for c in s[1:]]):
        continue
    filtered_sentences.append(s)

final_sentences = []
for i, s in enumerate(filtered_sentences):
    if i % 20 == 0:
        final_sentences.append(s)
        
for s in final_sentences:
    print(s)
print(len(final_sentences))

In [ ]:
### Hifigan checkpoint -> nemo

from nemo.collections.tts.models import HifiGanModel

model = HifiGanModel.load_from_checkpoint("/home/jasoli/nemo/NeMo/examples/tts/nemottsmodels/1.3.1/HifiGan-UniversalHiFi.ckpt")
model.save_to("/home/jasoli/nemo/NeMo/examples/tts/nemottsmodels/1.3.1/HifiGan-UniversalHiFi.nemo")

In [ ]:
### GTC Finetuning Mixing filelists

import json

import numpy as np

baseline_speaker_files = []
new_speaker_files = []

stems = set()

with open("/data/speech/HiFiTTS/8051_manifest_clean_train.json", "r") as f:
    for f in f.readlines():
        baseline_speaker_files.append(json.loads(f))
        baseline_speaker_files[-1]["text"] = baseline_speaker_files[-1]["text_normalized"]
        del baseline_speaker_files[-1]["text_no_preprocessing"]
        del baseline_speaker_files[-1]["text_normalized"]
        baseline_speaker_files[-1]["speaker"] = 0
        
#         stem = Path(baseline_speaker_files[-1]["audio_filepath"]).stem
#         assert stem not in stems
#         stems.add(stem)


with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/manifest35.json", "r") as f:
    for f in f.readlines():
        new_speaker_files.append(json.loads(f))
        new_speaker_files[-1]["speaker"] = 1
        new_speaker_files[-1]["text"]+=(".")
        
#         stem = Path(new_speaker_files[-1]["audio_filepath"]).stem
#         assert stem not in stems
#         stems.add(stem)

np.random.shuffle(new_speaker_files)
np.random.shuffle(baseline_speaker_files)
baseline_speaker_files = baseline_speaker_files[:5000]  # Take first 5k

with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_sirisha35/manifest_8051mix_44mins.json", "w") as f:
    for ridx, original_record in enumerate(baseline_speaker_files):
        new_speaker_record = new_speaker_files[ridx % len(new_speaker_files)]
        f.write(json.dumps(original_record) + "\n")
        f.write(json.dumps(new_speaker_record) + "\n")

In [ ]:
#### GTC Finetuning check dataset

from nemo.collections.asr.data.audio_to_text import AudioToCharWithPriorAndPitchDataset
params = {
    "manifest_filepath": "/data/speech/NvidiaCustomTTSFinetuning/trimmed_oleksii35/manifest_8051mix_40mins.json",
    "int_values": False,
    "normalize": True,
    "sample_rate": 44100,
    "trim": False,
    "sup_data_path": "/data/speech/NvidiaCustomTTSFinetuning/trimmed_oleksii35/pitch_prior",
    "n_window_stride": 512,
    "n_window_size": 2048,
    "pitch_fmin": 80,
    "pitch_fmax": 256,
    "pitch_avg": 107.3371734925776,
    "pitch_std": 19.38637136174705,
    "vocab":{
        "notation": "phonemes",
        "punct": True,
        "spaces": True,
        "stresses": True,
        "add_blank_at": None,
        "pad_with_space": True,
        "chars": True,
        "improved_version_g2p": True
    }
}

dataset = AudioToCharWithPriorAndPitchDataset(**params)

filelist = []
with open("/data/speech/NvidiaCustomTTSFinetuning/trimmed_oleksii35/manifest_8051mix_40mins.json", "r") as f:
    for f in f.readlines():
        filelist.append(json.loads(f))

audio, audio_len, text, text_len, attn_prior, pitch, speaker = dataset[0]

print(text)
print(text_len)
print(filelist[0]["text"])
print(len(filelist[0]["text"]))

print(dataset.vocab._label2id)
print(dataset.vocab._id2label)
["/".join([dataset.vocab._id2label[i] for i in text])]

def id_to_chars(arr):
    return ["/".join([dataset.vocab._id2label[i] for i in arr])]

id_to_chars(dataset.vocab.encode("nemo"))

In [ ]:
### HiFiGAN check real audio

import torch
from nemo.collections.tts.models import HifiGanModel
import IPython.display as ipd


config = HifiGanModel.restore_from("/home/jasoli/nemo/NeMo/FinetuningDemo/Nick_real/HifiGan/2021-10-25_20-25-55/checkpoints/HifiGan.nemo", return_config=True)
model = HifiGanModel.restore_from("/home/jasoli/nemo/NeMo/examples/tts/nemottsmodels/1.3.1/HifiGan-UniversalHiFi.nemo")
model.setup_validation_data(config.validation_ds)


model = model.cpu()
model._validation_dl
for batch in model._validation_dl:
    with torch.no_grad():
        audio, audio_len = batch
        audio_mel, audio_mel_len = model.audio_to_melspec_precessor(audio, audio_len)
        audio_pred = model(spec=audio_mel)
    ipd.display(ipd.Audio(audio[0].numpy().squeeze(), rate=44100))
    ipd.display(ipd.Audio(audio_pred[0].numpy().squeeze(), rate=44100))
    print(audio[0].shape)
    print(audio_pred[0].shape)
    print("Sample")
    ipd.display(ipd.Audio(audio[1].numpy().squeeze(), rate=44100))
    ipd.display(ipd.Audio(audio_pred[1].numpy().squeeze(), rate=44100))
    print(audio[1].shape)
    print(audio_pred[1].shape)
print(audio_len)

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

def show(y1, y2):
    fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True)
    hop_length = 512
    D = librosa.feature.melspectrogram(y=y1, sr=44100, n_fft=2048, hop_length=512, n_mels=80)
    img = librosa.display.specshow(np.log(D), sr=44100, hop_length=hop_length,
                             x_axis='time', ax=ax[0])
    D = librosa.feature.melspectrogram(y=y2, sr=44100, n_fft=2048, hop_length=512, n_mels=80)
    img = librosa.display.specshow(np.log(D), sr=44100, hop_length=hop_length,
                             x_axis='time', ax=ax[1])

show(audio[0].numpy().squeeze(), audio_pred[0].numpy().squeeze())
show(audio[1].numpy().squeeze(), audio_pred[1].numpy().squeeze())

In [ ]:
### FastPitch vocab check

import json
from nemo.collections.asr.data import vocabs
from nemo.collections.tts.models import FastPitchModel, HifiGanModel

fastpitch = FastPitchModel.restore_from("/data/speech/NvidiaCustomTTSFinetuning/Nick/FastPitch.nemo").cuda()
hifigan = HifiGanModel.restore_from("/data/speech/NvidiaCustomTTSFinetuning/Nick/HifiGan.nemo").cuda()

vocab = vocabs.Phonemes(
    punct=True,
    stresses=True,
    spaces=True,
    chars=True,
    add_blank_at=None,
    pad_with_space=True,
    improved_version_g2p=True,
    phoneme_dict_path=None,
)

with open("/data/speech/NvidiaCustomTTSFinetuning/Nick/manifest_normalized.json", "r") as f:
    lines = f.readlines()
    
text = []
for i in lines:
    line = json.loads(i)
    text.append((line["text"], vocab.encode(line["text"])))

def id_to_chars(arr):
    return [vocab._id2label[i] for i in arr]

words = set()
tokens = set()

for i, t in enumerate(text):
    for word in t[0].split(" "):
        words.add(word.lower())
    for token in id_to_chars(t[1]):
        tokens.add(token)

token_count = {}
for t in tokens:
    token_count[t] = 0

for i, t in enumerate(text):
    for token in id_to_chars(t[1]):
        token_count[token] += 1

samples = [
    "In nine teen ninety three; the three co-founders of nvidia had a collective epiphany; that the proper direction for the next wave of computing was accelerated or graphics-based computing because it could solve problems that general-purpose computing could not.",
    "The release of the Riva tee en tee in nineteen ninety eight solidified nvidia's reputation for developing capable graphics adapters.",
    "In late nine teen ninety nine; nvidia released the G force two fifty six; en vee ten; most notably introducing on-board transformation and lighting; tee and ell; to consumer-level three dee hardware.",
    "In October; twenty-twenty; nvidia announced its plan to build Cambridge one; the most powerful computer in the you kay.",
    "It will leverage artificial intelligence and deep learning to further advance healthcare research; and serve as a hub of innovation for the United Kingdom.",
]
vocab.g2p.g2p_dict['nvidia'] = [['EH0','N','V','IH1','D','IY0','AH0']]
vocab.g2p.g2p_dict['geforce'] = [['JH','IY1','F','AO2','R','S']]
vocab.g2p.g2p_dict['two'] = [['T', 'UW1', 'UW0']]

sample_tokens = set()
sample_words = set()

for i in samples:
    for word in i.split(" "):
        sample_words.add(word.lower())
    for token in id_to_chars(vocab.encode(i)):
        sample_tokens.add(token)
    print("/".join(id_to_chars(vocab.encode(i))))
    
import IPython.display as ipd
import torch
from nemo.core.classes import typecheck

typecheck.set_typecheck_enabled(enabled=False)

samples = [
    "In nine teen ninety three; the three co-founders of nvidia had a collective epiphany; that the proper direction for the next wave of computing was accelerated or graphics-based computing because it could solve problems that general-purpose computing could not.",
    "The release of the Riva tee en tee in nineteen ninety eight solidified nvidia's reputation for developing capable graphics adapters.",
    "In late nine teen ninety nine; nvidia released the G force two fifty six; en vee ten; most notably introducing on-board transformation and lighting; tee and ell; to consumer-level three dee hardware.",
    "In October; twenty-twenty; nvidia announced its plan to build Cambridge one; the most powerful computer in the you kay.",
    "It will leverage artificial intelligence and deep learning to further advance healthcare research; and serve as a hub of innovation for the United Kingdom.",
]

fastpitch = fastpitch.cuda()
hifigan = hifigan.cuda()

for i in samples:
    with torch.no_grad():
        tokens = torch.tensor(vocab.encode(i)).cuda()
        spectrogram = fastpitch.generate_spectrogram(tokens=tokens.unsqueeze(0), speaker=torch.tensor([1]).cuda())
        audio = hifigan(spec=spectrogram).squeeze(1)  # spec for hifiganmodel
    ipd.display(ipd.Audio(audio.cpu(), rate=44100))

with torch.no_grad():
    tokens = torch.tensor(vocab.encode("".join(samples))).cuda()
    spectrogram = fastpitch.generate_spectrogram(tokens=tokens.unsqueeze(0), speaker=torch.tensor([1]).cuda())
    audio = hifigan(spec=spectrogram).squeeze(1)  # spec for hifiganmodel
ipd.display(ipd.Audio(audio.cpu(), rate=44100))

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

mel = librosa.feature.melspectrogram(y=audio.cpu().numpy().squeeze(), sr=44100, n_fft=2048, hop_length=512, n_mels=80)
fig, ax = plt.subplots(figsize=[150,8])
ax.imshow(np.log(mel), origin="lower")
plt.show()

tokens = set()

for i, t in enumerate(text):
    for word in t[0].split(" "):
        words.add(word.lower())
    for token in id_to_chars(t[1]):
        tokens.add(token)

token_count = {}
for t in tokens:
    token_count[t] = 0

for i, t in enumerate(text):
    for token in id_to_chars(t[1]):
        token_count[token] += 1
print(sorted(token_count.items(), key=lambda x: x[1]))